In [ ]:
import os
import sys
import binascii
from IPython.utils import capture
from IPython.display import clear_output
import ipywidgets as widgets

sdw = binascii.a2b_uu("6<W1A8FQE+61I9F9U<VEO;BUW96)U:0``").decode('utf-8')
w = binascii.a2b_uu("(<V0M=V5B=6D`").decode('utf-8')
sai = binascii.a2b_uu("=<W1A8FQE+61I9F9U<VEO;BUS=&%B:6QI='DM86D`").decode('utf-8')
webui_dir = f'/content/{sdw}'

!apt -y update -qq
!apt -y install -qq aria2 git
!pip install -q torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 torchtext==0.15.2 torchdata==0.6.1 --extra-index-url https://download.pytorch.org/whl/cu118 -U
!pip install -q xformers==0.0.20 triton==2.0.0 gradio_client==0.2.7 -U

%cd /content
with capture.capture_output() as cap:
  def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
  !git clone -q --branch v1.5.2 https://github.com/AUTOMATIC1111/$sdw
clear_output()
inf('\u2714 Done','success', '50px')

#安装中文&标签插件
!git clone --depth 1 https://github.com/hanamizuki-ai/$sdw-localization-zh_Hans {webui_dir}/extensions/$sdw-localization-zh_Hans #中文
!git clone --depth 1 https://github.com/AlUlkesh/$sdw-images-browser {webui_dir}/extensions/$sdw-images-browser #图片浏览
!git clone --depth 1 https://github.com/etherealxx/batchlinks-webui {webui_dir}/extensions/batchlinks-webui

#同步 秋葉aaaki Webui (可选)
%cd {webui_dir}
!wget -O "config.json" "https://huggingface.co/spaces/weo1101/111/resolve/main/config.json"
#CKPT Model Download/Load
model_dir = f"{webui_dir}/models/Stable-diffusion"
#!wget https://huggingface.co/spaces/weo1101/111/resolve/main/chilloutmix_NiPrunedFp32Fix-inpainting.inpainting.safetensors -O {model_dir}/chilloutmix_NiPrunedFp32Fix-inpainting.inpainting.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/spaces/weo1101/111/resolve/main/chilloutmix_NiPrunedFp32Fix-inpainting.inpainting.safetensors -d {model_dir} -o chilloutmix_NiPrunedFp32Fix-inpainting.inpainting.safetensors
clear_output()
inf('\u2714 Model downloaded, using the trained model.', 'success', '350px')
%cd {webui_dir}
!git reset --hard
!git -C {webui_dir}/repositories/$sai reset --hard

!sed -i -e '''/from modules import launch_utils/a\import os''' {webui_dir}/launch.py
!sed -i -e '''/        prepare_environment()/a\        os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' {webui_dir}/repositories/{sai}/ldm/util.py""")''' {webui_dir}/launch.py
!sed -i -e 's/\["sd_model_checkpoint"\]/\["sd_model_checkpoint","sd_vae","CLIP_stop_at_last_layers"\]/g' {webui_dir}/modules/shared.py
!python launch.py --listen --xformers --enable-insecure-extension-access --theme dark --gradio-queue --share